In [1]:
import sys , os
from pathlib import Path
from importlib import reload

current_path = str(Path( os.getcwd()).parent.absolute())
os.chdir(current_path)

sys.path.append(current_path)
import app

In [ ]:
{
    "name":"Shoe",
    "price":500,
    "attributes":{
        "color":"black",
        "size" : "39-42",
        "material":"Leather",
        "brand":"Nike"
    }
}

In [2]:
from app.models.custom_vendor_plans import CustomVendorPlan

In [3]:
from app.models.model_utils import ObfuscateId
t = ObfuscateId.encode(1)
print(t)

ObfuscateId.decode(t)

Oejne5


1

In [4]:
from app.models import Charge
from app.models import Vendor , Product
from app.routes.routes_utils import db_session

from app.models.charges_utils import PayeeConstructor

In [ ]:
d = CustomVendorPlan(vendor_id=2 , base_plan_id=1)
db_session.add(d)
db_session.commit()

In [ ]:
d.product_commission_percent = 2
db_session.commit()

In [ ]:
d.vendor

In [ ]:
from app.data_manager.cart_manager import OrderManager

In [5]:
vendor = db_session.query(Vendor).filter_by(id=1).first()
prod = db_session.query(Product).filter_by(id=12).first()


prod.final_price ,prod.price ,vendor.plan.product_commission(prod.price) ,vendor.plan

(Decimal('172.00000'),
 Decimal('100.00'),
 120,
 <VendorPlan(id=1, name='TechMaster', commission=15.0%, flat_fee=120, min_price=800, payout=MONTHLY, min_payout=350)>)

In [ ]:
vendor.plan.products_min_price_threshold = 800
db_session.commit()

In [ ]:
vendor.products.all()

In [7]:
print(f"I am vendor:\n{vendor}\nMy plan is: {vendor.plan}\n")
for prod in vendor.products.all():
    print("-"*4)
    print(f"I am prod {prod} ")
    if prod.vendor_cover:
        print(f"My charges are {prod.charge.charge}\n{prod.vendor_cover.charge}")
    else:
         print(f"My charges are {prod.charge.charge}")

 

I am vendor:
<Vendor(id=1, name='TechMaster', email='tech@example.com', phone='1234567001', store_name='Tech Haven', payment_type='pre', verified=True)>
My plan is: <VendorPlan(id=1, name='TechMaster', commission=15.0%, flat_fee=120, min_price=800, payout=MONTHLY, min_payout=350)>

----
I am prod <Product(id=1, name='Wireless Bluetooth Earbuds', vendor_id=1,
base price = 59.99 selling price=167.99000, stock=150
total commission=120 ,vendor charges = 12.00000 ,product charges=108.00000)> 
My charges are <Charge 2: Type=PRODUCT, Percentage=90.00%, Recipient=1, Payee=Payee(entity_type='product', entity_id=1)>
<Charge 1: Type=PRODUCT, Percentage=10.00%, Recipient=1, Payee=Payee(entity_type='vendor', entity_id=1)>
----
I am prod <Product(id=2, name='Smart Watch Pro', vendor_id=1,
base price = 199.99 selling price=319.99000, stock=75
total commission=120 ,vendor charges = 0 ,product charges=120.00000)> 
My charges are <Charge 3: Type=PRODUCT, Percentage=100.00%, Recipient=2, Payee=Payee(enti

In [ ]:
prod = db_session.query(Product).filter_by(id=15).first()


prod.vendor.plan.product_commission(prod.price)
from decimal import Decimal
assert type(prod.vendor_commision) == Decimal
assert type(prod.commission) == Decimal

prod.vendor_commision  +  prod.commission ,prod.total_commission

In [ ]:
prod.price ,prod.final_price ,prod

In [ ]:
prod.vendor.plan.products_min_price_threshold

In [ ]:
vendor.plan.products_min_price_threshold =70
vendor.plan.product_commission_percent = 15
db_session.commit()

In [ ]:
from app.data_manager.charges_manager import ChargeRecorder
from app.seeds.platform_seeds import get_plaform_id
ChargeRecorder.record_products_charges(
    1
)

In [ ]:
ChargeRecorder.calculate_vendor_product_fee(
    db_session=db_session,
    product=10
)

In [ ]:
from app.data_manager.vendor_transaction import VendorTransactionSystem as VTM 

# VTM.update_vendors_accounts({1:87}) 
_,sales_data = VTM.divide_order_to_vendors(order_id=1)
print(sales_data)

session = {"session_token":"TestToken"}

fee_data=ChargeRecorder.record_products_charges(
    order_id=1,
    session_tkn=session['session_token']) 
## Commission charges on products
commission_data = ChargeRecorder.record_products_commision(
   1,
    session_tkn=session['session_token'])

net_sales  ={}
for vendor,amount in sales_data.items():

    net_sales[vendor] = sum(
        [
            amount,
            0-fee_data.get(vendor ,0),
            0-commission_data.get(vendor,0)
        ]
    )

net_sales[get_plaform_id()]= sum(fee_data.values()) + sum(commission_data.values())
net_sales
# VTM.update_vendors_accounts(sales_data)

In [ ]:
0-89

In [ ]:
session = {"session_token":"TestToken"}

fee_data=ChargeRecorder.record_products_charges(
    order_id=1,
    session_tkn=session['session_token'])
commission_data = ChargeRecorder.record_products_commision(
   1,
    session_tkn=session['session_token'])
total_fee = sum(fee_data.values()) + sum(commission_data.values())

In [ ]:

commission_data = dict([(id_,charge*-1) for id_,charge in commission_data.items()])
fee_data = dict([(id_,charge*-1) for id_,charge in fee_data.items()])

for vendor in commission_data:
    if vendor in fee_data:
        fee_data[vendor]+=commission_data[vendor]
    else:
        fee_data[vendor]=commission_data[vendor]
fee_data[get_plaform_id()]=total_fee

fee_data

In [ ]:
ChargeRecorder.record_products_commision(1)

In [ ]:
prod =db_session.query(Product).filter_by(id=12).first()
plan = prod.vendor.plan

prod.commission

In [ ]:
normal_withdrawal_charge = Charge(
    type=ChargeType.WITHDRAWAL_NORMAL,
    percentage = vendor.plan.scheduled_withdrawal_percentage,
    recipient_id = vendor.plan_id
)

emergency_withdrawal_charge = Charge(
    type=ChargeType.WITHDRAWAL_UNSCHEDULED,
    percentage = vendor.plan.unsheduled_withdrawal_percentage,
    recipient_id = vendor.plan_id
)

In [ ]:
paye= PayeeConstructor("product",4)

In [ ]:
with session_scope() as db_session:
    c = CM.get_charges(db_session ,payee=paye)
c

In [ ]:
# charge = CM.create_charge(ChargeType.PRODUCT,50,4,Payee('vendor',1))
charge = CM.create_charge(ChargeType.PRODUCT,50,3,Payee('vendor',1))

In [ ]:
plan =db_session.query(VendorPlan).filter_by(id=2).first()
plan

In [ ]:
from datetime import datetime
from app.models.model_utils import  PayoutFrequency


today = datetime.now().date().day 

withdrawal_freq = vendor.plan.payoutfrequency.BI_WEEKLY

if withdrawal_freq == PayoutFrequency.BI_WEEKLY:
    verified = today %14 in [1,13,0]
elif withdrawal_freq == PayoutFrequency.MONTHLY:
    verified = today %28 in [1,2,3,27,26,25,0]

elif withdrawal_freq  == PayoutFrequency.WEEKLY:
    verified = today % 7 in [1,2,6 ,0]
elif withdrawal_freq == PayoutFrequency.THREE_DAY_INTERVAL:
    veirifed = not today %3 
else:
    raise Exception("Unhandled withdrawall frequency logic")

if verified:
    charge_type = ChargeType.WITHDRAWAL_NORMAL

print(verified)

In [ ]:
v=db_session.query(Vendor).filter(Vendor.id==1).first()
v.charges[0].charge

In [ ]:
v.products.limit(2).all()

### WORKING WITH ```onupdate``` arguement in sqlachemy columns


In [ ]:
from sqlalchemy.orm import sessionmaker ,relationship  ,declarative_base
from sqlalchemy import( engine ,Column,Integer , 
                       String ,create_engine ,ForeignKey ,
                         TIMESTAMP ,Table ,Numeric,Enum 
                         ,DECIMAL)
import json
from datetime import datetime ,timezone
import random
from sqlalchemy.ext.hybrid import hybrid_property
from app.models.charges_utils import MutablePayee,Payee


BaseC = declarative_base()

class Base(BaseC):
    __abstract__ = True

    created_at = Column(TIMESTAMP, default=datetime.now(timezone.utc))
    updated_at = Column(TIMESTAMP,default=datetime.now(timezone.utc) ,onupdate=datetime.now(timezone.utc))


tag_book = Table(
    "tags_books",
    Base.metadata,
    Column("tag_id",Integer,ForeignKey("tags.id")),
    Column("book_id",Integer ,ForeignKey("books.id"))
    
)
tag_author = Table(
    'tags_authors',
    Base.metadata,
    Column('author_id',Integer , ForeignKey('authors.id')),
    Column("tag_id",Integer,ForeignKey("tags.id"))
)
charges_books = Table(
    'charges_book_charges',
    Base.metadata,
    Column('charge',Integer , ForeignKey('charges.id')),
    Column("book_charge",Integer,ForeignKey("books_charges.id"))
)



class AuthorBalance(Base):
    __tablename__ = "balances"
    id = Column(Integer,primary_key=True,autoincrement=True)
    balance = Column(DECIMAL ,nullable=False)

    author_id = Column(Integer , ForeignKey('authors.id') ,nullable=False)
    author = relationship('Author',back_populates='balance',uselist=False)

class Transaction(Base):
    __tablename__ = "transactions"
    id = Column(Integer,primary_key=True,autoincrement=True)

    source = Column(String ,nullable = False)
    recepient = Column(String , nullable = False)
    amount = Column(DECIMAL , nullable = False)

class BookSale(Base):
    __tablename__ = "books_sales"
    id = Column(Integer,primary_key=True , autoincrement=True)
    book_id = Column(Integer,ForeignKey("books.id"))
    
    author_id = Column(Integer , ForeignKey('authors.id'),nullable=False)
    transaction_id = Column(Integer , ForeignKey("transactions.id"))

    author = relationship("Author",backref='books_sold')
    book = relationship('Book',backref='purchases')
    transaction = relationship("Transaction",backref='book_sale',uselist=False)

class Tag(Base):
    __tablename__ = "tags"
    id = Column(Integer ,primary_key = True,autoincrement=True)
    name = Column(String)

    books = relationship('Book',back_populates='tags',secondary=tag_book)
    authors = relationship('Author',back_populates='tags',secondary=tag_author)

    def __repr__(self):
        return f"<{self.name}>"

class Book(Base):
    __tablename__ = "books"

    id = Column(Integer ,primary_key = True,autoincrement=True)
    title = Column(String)
    pages = Column(Integer)
    base_price = Column(Numeric(10, 2),default=round(random.uniform(9.99, 39.99), 1))  

    author_id = Column(Integer, ForeignKey('authors.id'), nullable=False)

    tags = relationship("Tag",back_populates='books',secondary=tag_book)
    author = relationship('Author', back_populates='books')
    charge = relationship('BookCharge', back_populates='book',uselist=False)

    @hybrid_property
    def price(self):
        if not self.charge:
            return self.base_price
        
        final_price = self.base_price *(1+ self.charge.percentage/100)
        return final_price
  
    def __repr__(self):
        return f"<(id={self.id} ,title={self.title} ,pages={self.pages})>"
    


class Author(Base):
    __tablename__ = "authors"
    id = Column(Integer ,primary_key = True,autoincrement=True)
    name = Column(String)
    age = Column(Integer)

    books = relationship('Book',back_populates='author')
    tags = relationship('Tag',back_populates='authors',secondary=tag_author)
    
    balance = relationship('AuthorBalance',back_populates='author',uselist=False)
    charge_rules = relationship('AuthorCharge',backref='author',lazy='dynamic')

    def __repr__(self):
        return f"<(id={self.id} ,name={self.name} ,num-books={len(self.books)})>"

class Charge(Base):
    __tablename__ = 'charges'
    
    id = Column(Integer, primary_key=True)
    type = Column(Enum(ChargeType), nullable=False)
    percentage = Column(Numeric(5, 2))
    recipient_id = Column(Integer)

    payee_id = composite(
            MutablePayee,
            Column('payee_type', String(20)),
            Column('payee_id', Integer)     
        )

    def __str__(self):
        return f"<Charge {self.id}: Type={self.type.name}, Percentage={self.percentage}%, Recipient={self.recipient_id}, Payee={self.payee_id}>"

    def __repr__(self):
        return (
            f"Charge(id={self.id}, "
            f"type={self.type.name}, "
            f"percentage={self.percentage}, "
            f"recipient_id={self.recipient_id}, "
            f"payee_id={self.payee_id})"
        )


class BookCharge(Base):
    __tablename__ = 'books_charges'
    id = Column(Integer ,primary_key=True,autoincrement=True)
    book_id = Column(Integer,ForeignKey('books.id') ,nullable=False)
    charge_id = Column(Integer ,ForeignKey('charges.id'),nullable=False)

    book = relationship('Book', back_populates='charge',uselist=False)
    charges=relationship("Charge" ,lazy='dynamic',secondary = charges_books ,uselist=True)

    @hybrid_property
    def percentage(self):
        return self.charge.percentage
    
    def __repr__(self):
        return f"<(charge={self.charge})"
    
class AuthorCharge(Base):
    __tablename__ = 'withdrawal_charges'
    id = Column(Integer ,autoincrement=True,primary_key=True)
    author_id = Column(Integer , ForeignKey('authors.id'))
    charge_id = Column(Integer,ForeignKey('charges.id'))

  



In [ ]:
from decimal import Decimal, ROUND_HALF_UP ,ROUND_UP

num = Decimal("7.321")
rounded_num = num.quantize(Decimal("0.01"))
print(rounded_num)  # Output: 7.33

In [ ]:
from sqlalchemy.orm import sessionmaker ,relationship  ,declarative_base
from sqlalchemy import( engine ,Column,Integer , 
                       String ,create_engine ,ForeignKey ,
                         TIMESTAMP ,Table ,Numeric,Enum 
                         ,DECIMAL)
import json
from datetime import datetime ,timezone
import random
from sqlalchemy.ext.hybrid import hybrid_property


BaseC = declarative_base()

class Base(BaseC):
    __abstract__ = True

    created_at = Column(TIMESTAMP, default=datetime.now(timezone.utc))
    updated_at = Column(TIMESTAMP,default=datetime.now(timezone.utc) ,onupdate=datetime.now(timezone.utc))


tag_book = Table(
    "tags_books",
    Base.metadata,
    Column("tag_id",Integer,ForeignKey("tags.id")),
    Column("book_id",Integer ,ForeignKey("books.id"))
    
)
tag_author = Table(
    'tags_authors',
    Base.metadata,
    Column('author_id',Integer , ForeignKey('authors.id')),
    Column("tag_id",Integer,ForeignKey("tags.id"))
)
charges_books = Table(
    'charges_book_charges',
    Base.metadata,
    Column('charge',Integer , ForeignKey('charges.id')),
    Column("book_charge",Integer,ForeignKey("books_charges.id"))
)



class AuthorBalance(Base):
    __tablename__ = "balances"
    id = Column(Integer,primary_key=True,autoincrement=True)
    balance = Column(DECIMAL ,nullable=False)

    author_id = Column(Integer , ForeignKey('authors.id') ,nullable=False)
    author = relationship('Author',back_populates='balance',uselist=False)

class Transaction(Base):
    __tablename__ = "transactions"
    id = Column(Integer,primary_key=True,autoincrement=True)

    source = Column(String ,nullable = False)
    recepient = Column(String , nullable = False)
    amount = Column(DECIMAL , nullable = False)

class BookSale(Base):
    __tablename__ = "books_sales"
    id = Column(Integer,primary_key=True , autoincrement=True)
    book_id = Column(Integer,ForeignKey("books.id"))
    
    author_id = Column(Integer , ForeignKey('authors.id'),nullable=False)
    transaction_id = Column(Integer , ForeignKey("transactions.id"))

    author = relationship("Author",backref='books_sold')
    book = relationship('Book',backref='purchases')
    transaction = relationship("Transaction",backref='book_sale',uselist=False)

class Tag(Base):
    __tablename__ = "tags"
    id = Column(Integer ,primary_key = True,autoincrement=True)
    name = Column(String)

    books = relationship('Book',back_populates='tags',secondary=tag_book)
    authors = relationship('Author',back_populates='tags',secondary=tag_author)

    def __repr__(self):
        return f"<{self.name}"


from sqlalchemy import case, select ,Float 
from sqlalchemy  import func

class Book(Base):
    __tablename__ = "books"

    id = Column(Integer ,primary_key = True,autoincrement=True)
    title = Column(String)
    pages = Column(Integer)
    base_price = Column(Numeric(10, 2),default=round(random.uniform(9.99, 39.99), 1))  

    author_id = Column(Integer, ForeignKey('authors.id'), nullable=False)

    tags = relationship("Tag",back_populates='books',secondary=tag_book)
    author = relationship('Author', back_populates='books')
    charges = relationship('BookCharge', back_populates='book',lazy='dynamic')

    @hybrid_property
    def price(self):
        """Calculate final price including any BOOK-paid commissions.
        Uses Python-side filtering for better readability with small datasets."""
        for charge_rel in self.charges:
            if charge_rel.charge.payee_id == f"BOOK-{self.id}":
                return self.base_price * (1 + charge_rel.charge.percentage / 100)
        return self.base_price  # Default if no matching charge
    

    
    def __repr__(self):
        return f"<(id={self.id} ,title={self.title} ,pages={self.pages} ,base_price={self.base_price} ,net_cost={self.price})>"
    


class Author(Base):
    __tablename__ = "authors"
    id = Column(Integer ,primary_key = True,autoincrement=True)
    name = Column(String)
    age = Column(Integer)

    books = relationship('Book',back_populates='author')
    tags = relationship('Tag',back_populates='authors',secondary=tag_author)
    
    balance = relationship('AuthorBalance',back_populates='author',uselist=False)
    charge_rules = relationship('AuthorCharge',backref='author',lazy='dynamic')

    def __repr__(self):
        return f"<(id={self.id} ,name={self.name} ,num-books={len(self.books)})>"



class Charge(Base):
    __tablename__ = 'charges'
    
    id = Column(Integer, primary_key=True)
    type = Column(Enum(ChargeType), nullable=False)
    percentage = Column(Numeric(5, 2))
    recipient_id = Column(Integer)
    payee_id = Column(String)

    def __str__(self):
        return f"<Charge {self.id}: Type={self.type.name}, Percentage={self.percentage}%, Recipient={self.recipient_id}, Payee={self.payee_id}>"

    def __repr__(self):
        return (
            f"Charge(id={self.id}, "
            f"type={self.type.name}, "
            f"percentage={self.percentage}, "
            f"recipient_id={self.recipient_id}, "
            f"payee_id={self.payee_id})"
        )



def construct_payee_id(id_:int):
    return f'BOOK-{id_}'

class BookCharge(Base):
    __tablename__ = 'books_charges'
    id = Column(Integer ,primary_key=True,autoincrement=True)
    book_id = Column(Integer,ForeignKey('books.id') ,nullable=False)
    charge_id = Column(Integer ,ForeignKey('charges.id'),nullable=False)

    charge=relationship("Charge",backref='book_charge',uselist=False)
    book = relationship('Book', back_populates='charges')

    @hybrid_property
    def percentage(self):
        return self.charge.percentage
    
    def __repr__(self):
        return f"<(Bookcharge={self.charge})"
    
class AuthorCharge(Base):
    __tablename__ = 'withdrawal_charges'
    id = Column(Integer ,autoincrement=True,primary_key=True)
    author_id = Column(Integer , ForeignKey('authors.id'))
    charge_id = Column(Integer,ForeignKey('charges.id'))

  



In [ ]:
engine = create_engine("sqlite:///bookstore.db")
Base.metadata.create_all(bind=engine)

In [ ]:
engine.url

In [ ]:

ses = sessionmaker(bind=engine)

bok = ses().query(Book).filter_by(id=1).first()
bok

In [ ]:
books_ = ses().query(Book).filter(Book.id <4).all()
print(books_)

In [ ]:

import re

class DataSeeder:
    def __init__(self, session:Session):
        self.session = session
        self.commission = 3.5 
        
    def seed_charges(self, charges_data):
        """
        Creates charge rules with strict enforcement of business logic:
        
        1. Charge Application:
        - PRODUCT charges apply to books (recipient=book_id)
        - WITHDRAWAL charges apply to authors (recipient=author_id)
        
        2. Payee Determines Financial Responsibility:
        - BOOK-{id} → Cost built into product price (customer pays)
        - AUTHOR-{id} → Cost deducted from author earnings
        
        3. Ownership Validation:
        - For PRODUCT charges with AUTHOR payee: Must be book's owner
        - For WITHDRAWAL charges: Payee must match recipient author
        
        Args:
            charges_data: {
                "id": {
                    "type": ChargeType,
                    "percentage": float,
                    "payee": "AUTHOR-{id}" or "BOOK-{id}",
                    "recipient": book_id or author_id
                }
            }
        """
        for _, charge_data in charges_data.items():
            # Normalize inputs
            charge_type = charge_data['type']
            payee = charge_data['payee'].upper().strip()
            recipient = str(charge_data['recipient']).strip()
            percentage = float(charge_data['percentage'])

            # Check for duplicate rule
            if self.session.query(Charge).filter_by(
                type=charge_type,
                payee_id=payee,
                recipient_id=recipient
            ).first():
               continue 

            # Validate payee format
            if not re.fullmatch(r'(AUTHOR|BOOK)-\d+', payee):
                raise ValueError(f"Invalid payee format: {payee}")

            # PRODUCT charge validation
            if charge_type == ChargeType.PRODUCT:
                book = self.session.query(Book).filter_by(id=recipient).first()
                if not book:
                    raise ValueError(f"Book {recipient} not found")

                if 'AUTHOR' in payee:
                    author_id = int(payee.split('-')[1])
                    if author_id != book.author_id:
                        raise ValueError(
                            f"Author {author_id} doesn't own book {recipient}"
                        )

            # WITHDRAWAL charge validation
            elif charge_type == ChargeType.WITHDRAWAL_UNSCHEDULED or charge_type==ChargeType.WITHDRAWAL_NORMAL:
                if not payee == f"AUTHOR-{recipient}":
                    raise ValueError(
                        f"Withdrawal payee {payee} must match recipient {recipient}"
                    )
                if not self.session.query(Author).filter_by(id=recipient).first():
                    raise ValueError(f"Author {recipient} not found")

            # Create charge record
            charge = Charge(
                type=charge_type,
                percentage=percentage,
                payee_id=payee,
                recipient_id=recipient
            )
            self.session.add(charge)
            self.session.flush()

           
            if charge_type == ChargeType.PRODUCT:
          
                if not self.session.query(BookCharge).filter_by(
                    book_id=int(recipient),
                    charge_id=charge.id
                ).first():
                    self.session.add(BookCharge(
                        book_id=int(recipient),
                        charge_id=charge.id
                    ))
            else: 
                if not self.session.query(AuthorCharge).filter_by(
                    author_id=int(recipient),
                    charge_id=charge.id
                ).first():
                    self.session.add(AuthorCharge(
                        author_id=int(recipient),
                        charge_id=charge.id
                    ))
                    
        self.session.commit()
    
    
    def seed_tags(self, tags_data):
        """Seed tags into the database, skipping existing ones."""
        existing_tags = {tag.name: tag for tag in self.session.query(Tag).all()}
        new_tags = [Tag(name=name) for name in tags_data if name not in existing_tags]
        
        if new_tags:
            self.session.add_all(new_tags)
            self.session.commit()
        return {**existing_tags, **{tag.name: tag for tag in new_tags}}

    def seed_authors(self, authors_data, all_tags):
        """Seed authors and their books into the database."""
        for author_data in authors_data:
            author = self.session.query(Author).filter_by(name=author_data["name"]).first()
            
            if not author:
                author = Author(
                    name=author_data["name"],
                    age=author_data["age"],
                    balance=AuthorBalance(balance=0)
                )
                self.session.add(author)
                self.session.flush()

            # Process author's books
            for book_data in author_data["books"]:
                self._process_book(author, book_data, all_tags)

            # Update author tags
            book_tags = set()
            for book in author.books:
                book_tags.update(book.tags)
            
            current_tags = set(author.tags)
            new_tags = book_tags - current_tags
            if new_tags:
                author.tags.extend(new_tags)
        
        self.session.commit()

    def _process_book(self, author, book_data, all_tags):
        """Helper method to process individual books."""
        book = self.session.query(Book).filter_by(
            title=book_data["title"],
            author_id=author.id
        ).first()

        if not book:
            book = Book(
                title=book_data["title"],
                pages=book_data["pages"],
                author_id=author.id,
                base_price=round(random.uniform(9.99, 39.99), 2)
            )
            self.session.add(book)
            self.session.flush()

        # Update book tags
        book.tags = [all_tags[tag_name] for tag_name in book_data["tags"] if tag_name in all_tags]
        return book

    def sell_book(self, book_id, customer_id):
        """Process a book sale, updating balances and creating transactions."""
        book = self.session.query(Book).get(book_id)
        if not book:
            raise ValueError("Book not found")

        author = book.author
        if not author.balance:
            author.balance = AuthorBalance(balance=0)
            self.session.add(author.balance)
            self.session.flush()

        # Calculate amounts
        commission_amount = book.base_price * (self.commission / 100)
        author_amount = book.base_price - commission_amount

        # Create transaction
        transaction = Transaction(
            source=customer_id,
            recepient=str(author.id),
            amount=book.base_price
        )
        self.session.add(transaction)
        self.session.flush()

        # Record book sale
        sale = BookSale(
            book_id=book.id,
            author_id=author.id,
            transaction_id=transaction.id
        )
        self.session.add(sale)

        # Update balances
        author.balance.balance += author_amount
        self.session.commit()

        return {
            'book': book.title,
            'price': float(book.base_price),
            'author_earned': float(author_amount),
            'commission': float(commission_amount),
            'transaction_id': transaction.id
        }


    def simulate_withdrawal(self, author_id, amount):
        """Simulate a withdrawal from an author's balance."""
        author = self.session.query(Author).filter_by(id=author_id)
        if not author:
            raise ValueError("Author not found")

        if not author.balance:
            raise ValueError("Author has no balance account")

        if author.balance.balance < amount:
            raise ValueError("Insufficient funds")

        author.balance.balance -= amount
        self.session.commit()
        return {
            'author_id': author.id,
            'new_balance': float(author.balance.balance),
            'withdrawn': float(amount)
        }

    def populate_all(self, authors_data, tags_data):
        """Convenience method to populate all data at once."""
        all_tags = self.seed_tags(tags_data)
        print("Done seeding tags")
        self.seed_authors(authors_data, all_tags)

In [ ]:

Session = sessionmaker(bind=engine)
session= Session()
seeder = DataSeeder(session)
# Save this as data.json
# Then load and seed it:
with open('data.json') as f:
    data = json.load(f)
seeder.populate_all(data['authors'], data['tags'])


In [ ]:
!pwd

In [ ]:
## book covers commision
charge_data = {
    "1":{
        'type':ChargeType.PRODUCT,
        'percentage':3.4,
        "payee":'BOOK-1',
        "recipient":"1"
    }
}

seeder.seed_charges(charge_data)

## author covers commision
charge_data = {
    "1":{
        'type':ChargeType.PRODUCT,
        'percentage':3.4,
        "payee":'AUTHOR-1',
        "recipient":"2"
    }
}
seeder.seed_charges(charge_data)



In [ ]:
charge_data3 = {
    "1":{
        'type':ChargeType.PRODUCT,
        'percentage':1,
        "payee":'AUTHOR-2',
        "recipient":"6"
    }
}
seeder.seed_charges(charge_data3)

charge_data3 = {
    "1":{
        'type':ChargeType.PRODUCT,
        'percentage':2.4,
        "payee":'BOOK-6',
        "recipient":"6"
    }
}
seeder.seed_charges(charge_data3)

In [ ]:
bok = Session().query(Book).filter_by(id=6).first()

book_charges = bok
bok.price

In [ ]:
query_ses = Session()

query_ses.query(Author).get('1')

In [ ]:
author = query_ses.query(
    Author
).filter_by(id='1').first()
print(author)
# c = query_ses.query(Book).filter_by(id=2).first()

bok ,bok.author

In [ ]:
bok.price ,bok.base_price

In [ ]:
[9 ,8]-[9,0]